In [307]:
import os
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np

In [308]:
import re

In [309]:
#Read the files
base_path = "C:/Users/reema.alhenaki/Desktop/llama3_Data/data/split"
patient_df = pd.read_csv(os.path.join(base_path, "HIS_Patient.csv"))
vitals_df = pd.read_csv(os.path.join(base_path, "HIS_PatientVitalSigns.csv"))
appointments_df = pd.read_csv(os.path.join(base_path, "HIS_Appointment.csv"))
docOrders_df = pd.read_csv(os.path.join(base_path, "HIS_DoctorOrder.csv"))

In [310]:
rows_with_nulls = patient_df[patient_df.isnull().any(axis=1)]

In [311]:
patient_df= patient_df[["PatientID", "RegistrationDate", "FirstName", "MiddleName", "LastName", "Gender", "DateofBirth", 
                          "NationalityID", "FirstVisit", "LastVisit", "NoOfVisit", "MobileNumber", "EmailAddress", "IsPregnant",
                          "BloodGroup", "RHFactor", "RegisteredDoctor", "EmergencyContactName", "EmergencyContactNo"]]

In [312]:
patient_df.shape

(12, 19)

In [313]:
#These are the columns that are important to use in production.
#vitals_df= vitals_df[[ "PatientID", "WeightKg", "HeightCm", "BodyMassIndex", "TemperatureCelcius",
                      #"PulseBeatPerMinute", "RespirationBeatPerMinute",
                      #"BloodPressureLower", "BloodPressureHigher", "SAO2", "FIO2", "PainScore",
                      #"PainLocation", "PainDuration", "PainCharacter", "PainFrequency",
                      #"TriageCategory", "GCScore", "CreatedOn", "Consciousness", "SkinColor", "OxygenTherapy",
                      #"TypeofOxygenTherapy", "OxygenFlow", "UrineOP", "LevelOfConsciousness", "HEWSScore", "RequiredAction",
                      #"ActionTaken", "SPO2", "VitalSignType", "Remarks"]]

In [314]:
# These are the columns used in this stage (All NULL columns are removed)
vitals_df= vitals_df[[ "PatientID", "WeightKg", "HeightCm", "BodyMassIndex", "TemperatureCelcius",
                      "PulseBeatPerMinute", "RespirationBeatPerMinute",
                      "BloodPressureLower", "BloodPressureHigher", "SAO2", "FIO2", "PainScore",
                      "PainLocation", "PainDuration", "PainCharacter", "PainFrequency",
                      "TriageCategory", "GCScore", "CreatedOn"]]

In [315]:
vitals_df.shape

(12, 19)

In [316]:
appointments_df= appointments_df[[ "AppointmentNo", "AppointmentDate","PatientID",
                                  "ClinicID", "DoctorID", "StartTime", "EndTime","VisitType","VisitFor",
                                  "Notes", "IsVirtual"]]

In [317]:
appointments_df.shape

(12, 11)

In [318]:
docOrders_df= docOrders_df[["PatientID", "ActualOrderDate", "ActualOrderTime", "OrderNotes", "NursingNotes"]]

In [319]:
docOrders_df.shape

(12, 5)

In [320]:
# Clean and Check Data Types
def clean_table(df, date_columns):
    # Drop duplicates
    df = df.drop_duplicates()

    # Convert date columns to datetime
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Reset index just in case
    df = df.reset_index(drop=True)
    
    return df

# Calling clean_table function to clean the tables. I also passes the columns with date type to ensure they are converted to datetime.
patient_df = clean_table(patient_df, ["RegistrationDate","DateofBirth", "FirstVisit", "LastVisit"])
vitals_df = clean_table(vitals_df, ["CreatdedOn"])
appointments_df = clean_table(appointments_df, ["AppointmentDate", "StartTime", "EndTime"])
doc_orders_df = clean_table(doc_orders_df, ["ActualOrderDate", "ActualOrderTime"])

#Converting int columns to int to ensure they have the acuurate data type.
cols_to_convert_to_int = ["BloodGroup", "NoOfVisit", "RegisteredDoctor", "EmergencyContactNo"]

for col in cols_to_convert_to_int:
    patient_df[col] = patient_df[col].fillna(0).astype(int)


# Optional: View data types
print("\nPatient Data Types:\n", patient_df.dtypes)
print("\nVital Signs Data Types:\n", vitals_df.dtypes)
print("\nAppointments Data Types:\n", appointments_df.dtypes)
print("\nDoctor Notes Data Types:\n", doc_orders_df.dtypes)




Patient Data Types:
 PatientID                        int64
RegistrationDate        datetime64[ns]
FirstName                       object
MiddleName                      object
LastName                        object
Gender                           int64
DateofBirth             datetime64[ns]
NationalityID                   object
FirstVisit              datetime64[ns]
LastVisit               datetime64[ns]
NoOfVisit                        int64
MobileNumber                     int64
EmailAddress                    object
IsPregnant                       int64
BloodGroup                       int64
RHFactor                        object
RegisteredDoctor                 int64
EmergencyContactName            object
EmergencyContactNo               int64
dtype: object

Vital Signs Data Types:
 PatientID                     int64
WeightKg                    float64
HeightCm                    float64
BodyMassIndex               float64
TemperatureCelcius          float64
PulseBeatPerMinut

/var/folders/cr/0rt_gyk14t78fv5h2f5vhdp80000gn/T/ipykernel_86118/1989131842.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
/var/folders/cr/0rt_gyk14t78fv5h2f5vhdp80000gn/T/ipykernel_86118/1989131842.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [324]:
#This function removes all the numbers in the EmergencyContactName and fill the number in EmergencyContactNo.

def clean_emergency_contacts(df):
    # 1. Remove leading/trailing spaces and ensure strings
    df['EmergencyContactName'] = df['EmergencyContactName'].astype(str).str.strip()
    df['EmergencyContactNo'] = df['EmergencyContactNo'].astype(str).str.strip()

    # 2. Handle 'NULL', 'nan', empty strings, and '0' properly (set to NaN)
    df['EmergencyContactNo'] = df['EmergencyContactNo'].replace(['nan', 'NaN', 'NULL', '', '0'], pd.NA)

    # 3. Extract Saudi phone numbers from EmergencyContactName (starting with 05)
    extracted_numbers = df['EmergencyContactName'].str.extract(r'(05\d{8})')

    # 4. Fill missing EmergencyContactNo with extracted numbers
    df['EmergencyContactNo'] = df['EmergencyContactNo'].fillna(extracted_numbers[0])

    # 5. Remove phone numbers (with optional dash) from EmergencyContactName
    df['EmergencyContactName'] = df['EmergencyContactName'].str.replace(r'[-–]?\s*05\d{8}', '', regex=True).str.strip()

    # 6. Remove leading zero from EmergencyContactNo to start with 5
    df['EmergencyContactNo'] = df['EmergencyContactNo'].str.lstrip('0')

    # 7. Replace empty or 'nan' names with 'Unknown'
    df['EmergencyContactName'] = df['EmergencyContactName'].replace(['nan', 'NaN', 'NULL', ''], 'Unknown')

    return df


In [325]:
patient_df=clean_emergency_contacts(patient_df)

In [326]:
patient_df[['EmergencyContactNo','EmergencyContactName']] 


,EmergencyContactNo,EmergencyContactName
0,543334375,AHMAD
1,NaN,Unknown
2,561301110,FAHAD
3,503151048,ABDULLAH
4,566282670,TURKI
5,548444121,WAIF
6,547715000,RAGAD FOR BILING
7,543053466,SON
8,507064483,DRIVER
9,544265053,KALAD


In [327]:
import random

# Sample name pools
male_first_names = ["Ahmed", "Faisal", "Omar", "Yusuf", "Khalid"]
female_first_names = ["Aisha", "Fatimah", "Sara", "Noor", "Layla"]
middle_last_names = ["Hassan", "Abdullah", "Salem", "Nasser", "Mubarak"]

# Gender-aware first name assignment
def assign_first_name(gender):
    if gender == 1:
        return random.choice(male_first_names)
    elif gender == 2:
        return random.choice(female_first_names)
    else:
        return "Unknown"

# Apply name replacements
for col in ["FirstName", "MiddleName", "LastName"]:
    mask = (patient_df[col] == ".") | (patient_df[col].isna())
    if col == "FirstName":
        patient_df.loc[mask, col] = patient_df.loc[mask, "Gender"].apply(assign_first_name)
    else:
        patient_df.loc[mask, col] = [random.choice(middle_last_names) for _ in range(mask.sum())]

# Create email address from first name
patient_df["EmailAddress"] = patient_df["FirstName"].str.lower() + "@mail.com"


In [328]:
# Save Cleaned Files

output_path = os.path.join("C:/Users/reema.alhenaki/Desktop/llama3_Data/data/cleaned")
os.makedirs(output_path, exist_ok=True)

patient_df.to_csv(os.path.join(output_path, "HIS_Patient.csv"), index=False)
vitals_df.to_csv(os.path.join(output_path, "HIS_PatientVitalSigns.csv"), index=False)
appointments_df.to_csv(os.path.join(output_path, "HIS_Appointment.csv"), index=False)
docOrders_df.to_csv(os.path.join(output_path, "HIS_DoctorOrder.csv"), index=False)

print("All tables cleaned and saved successfully!")

All tables cleaned and saved successfully!
